In [8]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('myLogReg').getOrCreate()

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [ ]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
my_log_reg_model = LogisticRegression()

In [11]:
fitted_logreg =  my_log_reg_model.fit(my_data)

In [12]:
log_summary = fitted_logreg.summary

In [13]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [14]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [15]:
training_data, testing_data = my_data.randomSplit([0.7,0.3])

In [16]:
fitted_logreg =  my_log_reg_model.fit(training_data)

In [19]:
predictions_set = fitted_logreg.evaluate(testing_data)

In [20]:
predictions_set.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[18.4658535989544...|[0.99999999044167...|       0.0|
|  0.0|(692,[121,122,123...|[29.0110428289023...|[0.99999999999974...|       0.0|
|  0.0|(692,[123,124,125...|[38.2104661158570...|           [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[26.2079255321819...|[0.99999999999585...|       0.0|
|  0.0|(692,[125,126,127...|[38.4286915260759...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[34.1074529298884...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[42.4402720773354...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[30.8252387474630...|[0.99999999999995...|       0.0|
|  0.0|(692,[126,127,128...|[36.7204747362612...|[0.99999999999999...|       0.0|
|  0.0|(692,[151

In [21]:
from pyspark.ml.evaluation import ( BinaryClassificationEvaluator,MulticlassClassificationEvaluator)

In [22]:
my_eval = BinaryClassificationEvaluator()

In [23]:
my_final_roc = my_eval.evaluate(predictions_set.predictions)

In [24]:
my_final_roc

1.0